<a href="https://colab.research.google.com/github/nihermann/Pokemaenner/blob/main/Interactive_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title # Interactive Generator and Evolution Center.
import tensorflow as tf
import ipywidgets as widget
from IPython.display import (display, clear_output, Image)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import imageio
import random
import os
from google.colab import drive
drive.mount("/content/gdrive")

In [112]:
#@title ## Specify the Generator path
generator_path = "/content/gdrive/MyDrive/pokemaenner/AEGAN/g100.generator.h5" #@param {type:"string"}

slider = lambda x: widget.FloatSlider(
    value=1,
    min=-1,
    max=1,
    step=0.05,
    description=x,
    continuous_update=False,
)

def load_generator(path):
    gen = tf.keras.models.load_model(path, compile=False)
    ls = gen.input_shape[1]
    return gen, ls

generator, ls = load_generator(generator_path)

def get_image(vector):
    latent_vector = tf.expand_dims(tf.constant(vector), axis=0)
    return generator(latent_vector)

def show_image(placeholder):
    img = get_image([s.value for s in sliders])
    img = tf.cast((img[0] + 1) * 128, tf.uint8)
    plt.imshow(img)
    plt.axis("off")
    plt.show()

def interpolate(a, b, steps, apply_noise=True):
    step = (b - a) / steps
    for _ in range(steps-1):
        noise = tf.random.normal(a.shape, 0, 0.001)
        a += step if not apply_noise else step + noise
        yield a
    yield b

sliders = None
sliders = [slider(str(i+1)) for i in range(ls)]
for s in sliders:
    s.observe(show_image)

button = widget.Button(description="Random")
def random_init_slider(placeholder):
    for s in sliders:
        s.unobserve_all()
        s.value = random.gauss(0, 1)
        s.observe(show_image)

button.on_click(random_init_slider)
button.on_click(show_image)

In [ ]:
#@title # Display sliders
for s in sliders:
    display(s)

In [ ]:
#@title ## Random Assignment
display(button)
clear_output(wait=True)

In [ ]:
#@title Execute me to see an Evolution
smoothness = 35 #@param {type:"slider", min:2, max:35, step:1}
random_assignment = True #@param {type:"boolean"}
start_constant = -0.415 #@param {type:"slider", min:-1, max:1, step:0.005}
end_constant = 1 #@param {type:"slider", min:-1, max:1, step:0.005}
if random_assignment:
    a = tf.random.normal((ls,))
    b = tf.random.normal((ls,))
else:
    a = tf.ones((ls,)) * start_constant
    b = tf.ones((ls,)) * end_constant

filenames = []

for i, vec in enumerate(interpolate(a, b, smoothness)):
    img = get_image(vec)
    img = tf.cast((img[0] + 1) * 128, tf.uint8)
    plt.imshow(img)
    plt.axis("off")

    filename = f'{i+1}.png'
    filenames.append(filename)
    
    # save frame
    plt.savefig(filename)
    plt.close()

# build gif
with imageio.get_writer('mygif.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        
# Remove files
for filename in set(filenames):
    os.remove(filename)

Image(open('mygif.gif','rb').read())